In [1]:
#pip install tweepy
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import re
import string
import os
import time

In [2]:
#use config file with your API authentication details
from twitter_config import * 

In [6]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [7]:
#function to scrape

def scraptweets(search_words, date_until, date_since, numTweets, numRuns):
    
    
    
    # Define a pandas dataframe to store the date:
    db_tweets = pd.DataFrame(columns = ['userid','username', 'acctdesc', 'location', 'following',
                                        'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts',
                                        'retweetcount', 'text', 'hashtags','tweet_cursor_id']
                                )
    program_start = time.time()
    #maxid helps as a variable to prevent duplicated tweets when we run the function through diff. runs. 
    maxid=None 
    for i in range(0, numRuns):
        # We will time how long it takes to scrape tweets for each run:
        start_run = time.time()
        
        # Collect tweets using the Cursor object
        # .Cursor() returns an object that you can iterate or loop over to access the data collected.
        # Each item in the iterator has various attributes that you can access to get information about each tweet. estimated s'pore geocode
        tweets = tweepy.Cursor(api.search, q=search_words, lang="en", max_id=maxid, 
                               until=date_until,since=date_since, tweet_mode='extended').items(numTweets)
        # Store these tweets into a python list
        tweet_list = [tweet for tweet in tweets]
       
     
        noTweets = 0
        for tweet in tweet_list:
            # Pull the values
                userid=tweet.user.id
                username = tweet.user.screen_name
                acctdesc = tweet.user.description
                location = tweet.user.location
                following = tweet.user.friends_count
                followers = tweet.user.followers_count
                totaltweets = tweet.user.statuses_count
                usercreatedts = tweet.user.created_at
                tweetcreatedts = tweet.created_at
                retweetcount = tweet.retweet_count
                hashtags = tweet.entities['hashtags']
                temp_max_id=tweet.id
                text = tweet.full_text
                
                ith_tweet = [userid,username, acctdesc, location, following, followers, totaltweets,
                         usercreatedts, tweetcreatedts, retweetcount, text, hashtags,temp_max_id]
                # Append to dataframe - db_tweets
                db_tweets.loc[len(db_tweets)] = ith_tweet
                # increase counter - noTweets  
                noTweets += 1
                
        #find the max_id to feed into the arg for subsequent loops
        try:
            maxid= (tweet_list[-1].id - 1)
        except: 
            print("max tweets crawled")
        # Run ended:
        end_run = time.time()
        duration_run = round((end_run-start_run)/60, 2)
        
        print('no. of tweets scraped for run {} is {}'.format(i + 1, noTweets))
        print('time take for {} run to complete is {} mins'.format(i+1, duration_run))
        
        time.sleep(930) #15 minute sleep time to keep within api limit
    # Once all runs have completed, save them to a single csv file:
    from datetime import datetime
    
    # Obtain timestamp in a readable format
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
    # Define working path and filename
    path = os.getcwd()
    filename = path + to_csv_timestamp + '_wfh_tweets.csv'
    # Store dataframe in csv with creation date timestamp
    db_tweets.to_csv(filename, index = False)
    
    program_end = time.time()
    print('Scraping has completed!')
    print('Total time taken to scrap is {} minutes.'.format(round(program_end - program_start)/60, 2))

for the code below, selected search words are: 
- #workfromhome 
- #wfh 
- #LoveWhereverYouWork
- #AnotherDayAtTheHomeOffice
- #HomeSweetOffice
- #MyHomeOffice
- #SigningOnFromHome
- #MyWFHRoutine
- #SigningOnInSweatpants
- #LetsWorkFromHome
- #WorkFromWherever
- #WhenYouWorkFromHome
<br>
retweets are filtered out. 


In [8]:
# Initialise these variables: to crawl only one day tweet (T), date_until = T+1 and date_since=T. 
search_words = "#workfromhome OR #wfh OR #LoveWhereverYouWork OR #AnotherDayAtTheHomeOffice OR #HomeSweetOffice OR #MyHomeOffice OR #SigningOnFromHome OR #MyWFHRoutine OR #SigningOnInSweatpants OR #LetsWorkFromHome OR #WorkFromWherever OR #WhenYouWorkFromHome -filter:retweets "

#scrape tweets for 28 june
date_until = "2020-06-29"
date_since="2020-06-28"
numTweets = 2500
numRuns = 2
# Call the function scraptweets
scraptweets(search_words, date_until, date_since, numTweets, numRuns)

no. of tweets scraped for run 1 is 1386
time take for 1 run to complete is 2.4 mins
max tweets crawled
no. of tweets scraped for run 2 is 0
time take for 2 run to complete is 0.01 mins
Scraping has completed!
Total time taken to scrap is 33.416666666666664 minutes.
